In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

### link_list twitch metric

In [92]:
df_streamList = pd.DataFrame(columns = ["Name", "Link"])

In [95]:
for page in [1, 2]:
    rank_url = "https://www.twitchmetrics.net/channels/viewership?lang=ko&page=" + str(page)
    response = requests.get(rank_url)
    dom = BeautifulSoup(response.content, "html.parser")
    dom_stream_list = dom.select(".list-group .list-group-item .d-flex.mb-2.flex-wrap a")
    
    for dom_stream in dom_stream_list:
        link = dom_stream.get("href")
        name = dom_stream.text.strip().split(" ")[0]
        df_streamList.loc[len(df_streamList)] = {"Name" : name, "Link" : link}

In [97]:
df_streamList.head()

,Name,Link
0,풍월량,/c/103825127-hanryang1125
1,서새봄냥,/c/30904062-saddummy
2,따효니,/c/102381201-ddahyoni
3,overwatchleague_kr,/c/188864445-overwatchleague_kr
4,오버워치_이스포츠,/c/160075583-playoverwatch_kr


### get twitch metric info

In [137]:
stream_url = "https://www.twitchmetrics.net" + df_streamList["Link"][0]

##### Basic info

In [159]:
from selenium import webdriver

In [160]:
driver = webdriver.Chrome()

In [161]:
driver.get(stream_url)

In [172]:
stream_details = driver.find_elements_by_css_selector("div.card-block dd.col-6")

In [174]:
mature_content = stream_details[0].text
language 

'No'

In [152]:
response = requests.get(stream_url)
dom = BeautifulSoup(response.content, "html.parser")

In [153]:
names = dom.select_one(".media-body .text-white").text.split()
stream_name = names[0]
if len(names) == 2:
    stream_id = names[1][1:-1] 

In [154]:
stream_name, stream_id

('풍월량', 'hanryang1125')

### JSON info

##### recent viewship values

In [138]:
stream_recent_view_url = stream_url + "/recent_viewership_values"

In [142]:
stream_recent_view = requests.get(stream_recent_view_url)
stream_recent_view = stream_recent_view.json()

##### history values

In [146]:
stream_history_values_url = stream_url + "/stream_history_values"

In [147]:
stream_history_values = requests.get(stream_history_values_url)
stream_history_values = stream_history_values.json()

- 방송 시작 시간
- 방송 종료 시간
- 방송 종류
- 방송 시간(second)
- viewer(average)
- total followers
- total viewer